# Building, evaluating and fine-tuning model

## 1. Configuring Language and Retrieval model

In [1]:
import ollama
from qdrant_client import QdrantClient

import json

import dspy
import dspy.evaluate
from dspy.evaluate.evaluate import Evaluate
from dspy import teleprompt

import rag

/opt/anaconda3/envs/rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Read parameters file

parameters_file = "params.json"

with open(parameters_file, 'r') as fh:
    params = json.loads(fh.read())

In [3]:
# Load the Ollama local language model
lm = dspy.OllamaLocal(model=params["lm_name"])

# Connection to Qdrant Vector DB
client = QdrantClient("localhost", port=6333)

# Configuring language model and retieval model to use in dspy
dspy.settings.configure(lm=lm)

## Basic RAG model without prompt optimization

In [4]:
# Create RAG object
uncompiled_rag = rag.RAG()

## Queries

In [5]:
# Load Q/A data from json file
with open(params["qa_filename"], 'r') as fh:
    qa_data = json.loads(fh.read())

In [6]:
question = qa_data[0]['question']
result = uncompiled_rag(question)

print(result.context)
print(result.answer)

In [7]:
question = qa_data[1]['question']
result = uncompiled_rag(question)

print(result.context)
print(result.answer)

In [8]:
question = qa_data[2]['question']
result = uncompiled_rag(question)

print(result.context)
print(result.answer)

## Evaluate model

In [9]:
# Create test set from the Q/A data
test_set = [dspy.Example(question=x['question'], answer=x['answer']).with_inputs('question') for x in qa_data]

In [10]:
# Print model response for Q/A pairs 
 
# for doc in qa_data:
#     result = uncompiled_rag(doc['question'])
#     example = dspy.Example(question=doc['question'], answer=doc['answer']).with_inputs('question')
#     pred = dspy.Example(question=doc['question'], answer=result["answer"]").with_inputs('question')
#     print('Q: ' + doc['question'])
#     print('A: ' + doc['answer'])
#     print('SA: ' + result["answer"])
#     print('Score: ' + rag.MetricWrapper(example, pred))
#     print('\n')
#     print('Context:')
#     print(result["context"])


In [11]:
# Evaluate model using the Q/A dataset and metric

evaluate_on_qa = Evaluate(devset=test_set, num_threads=4, display_progress=True, display_table=len(test_set))

evaluate_on_qa(uncompiled_rag, metric=rag.MetricWrapper)

Average Metric: 108.0 / 208  (51.9): 100%|██████████| 208/208 [38:37<00:00, 11.14s/it]


,question,example_answer,context,pred_answer,MetricWrapper,answer
0,What is Student Ambassador Program at TBC College?,"The Student Ambassador Program at TBC College provides friendly and respectful peer support for students, utilizing the lived experience of international students to help build...","[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","The Student Ambassador Program at the referenced college is a peer support system for international students, providing assistance in various areas such as homesickness, academic...",✔️ [True],nan
1,How can I access more information about TBC College's Student Ambassador Program?,You can find more information by clicking on this link: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg,"[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","To access more information about TBC College's Student Ambassador Program, click on this link: [Student Ambassador Program](https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg). This link will provide you with detailed information...",✔️ [True],nan
2,What is TBC College's approach towards student support?,"TBC College provides peer support, which recognizes that students naturally turn to each other for support and connection. Student Ambassadors utilize their lived experience as...","[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","TBC College's approach towards student support is centered around peer-to-peer interaction, where Student Ambassadors, who are international students themselves, provide friendly and respectful support to...",✔️ [True],nan
3,How can I access more information about TBC College's student support program?,You can click here: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg for more details.,"[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","To access more information about TBC College's student support programs, you can visit their official website at [http://www.services.tbcollege.com](http://www.services.tbcollege.com). From there, navigate to the ""Services"" section...",False,nan
4,What tutoring services does TBC College offer for enrolled students?,TBC College offers free tutoring services to all enrolled students. These services could be peer-led or faculty-led.,"[' Tutoring Services Students who need help with their course material, LCIT offers free tutoring services to all enrolled students. Tutoring services could be peer-led...","Based on the provided information, it appears that TBC College does not explicitly mention any tutoring services for enrolled students. However, considering the context of...",False,nan
5,How can I request tutoring services at TBC College?,"To request tutoring services, please fill out the Tutor Request Form available here: https://forms.office.com/pages/responsepage.aspx?id=PlKz1i9IoEKTulrUSLtK5CmSQlqeXtROpTBCYMDn29lUMDBRQjYxMkhWSlkzOFBRS1NGR1RaNVQ1RSQlQCN0PWcu","[' Tutoring Services Students who need help with their course material, LCIT offers free tutoring services to all enrolled students. Tutoring services could be peer-led...","To request tutoring services at TBC College, follow these steps: 1. Visit the college's official website (http://www.tbcollege.com). 2. Navigate to the ""Services"" section in the...",✔️ [True],nan
6,How can I register with Accessibility Services at TBC College?,"To register with Accessibility Services, please fill out the Accessibility Service Application available at this link: https://forms.office.c

51.92

In [12]:
# Initializing model optimizer

teleprompter = teleprompt.BootstrapFewShot(
    metric=rag.MetricWrapper,
    max_bootstrapped_demos=4, 
    max_labeled_demos=30)


In [13]:
# teleprompter = teleprompt.BootstrapFewShotWithRandomSearch(
#     metric=rag.MetricWrapper,
#     max_bootstrapped_demos=4, 
#     max_labeled_demos=4,
#     num_candidate_programs=5,
#     num_threads=8)

In [17]:
# Optimizing model prompt and weights

compiled_rag = teleprompter.compile(rag.RAG(), trainset=test_set)

  3%|▎         | 6/208 [01:55<1:04:37, 19.19s/it]

Bootstrapped 4 full traces after 7 examples in round 0.


In [15]:
# Print model response for Q/A pairs 

# for doc in qa_data:
#     result = compiled_rag(doc['question'])
#     example = dspy.Example(question=doc['question'], answer=doc['answer']).with_inputs('question')
#     pred = dspy.Example(question=doc['question'], answer=result["answer"]).with_inputs('question')
#     print('Q: ' + doc['question'])
#     print('A: ' + doc['answer'])
#     print('SA: ' + result["answer"])
#     print('Score: ' + rag.MetricWrapper(example, pred))
#     print('Context:')
#     print(result["context"])
#     print('\n')

In [18]:
# Evaluate fine-tuned model

evaluate_on_qa = Evaluate(devset=test_set, num_threads=8, display_progress=True, display_table=len(test_set))

evaluate_on_qa(compiled_rag, metric=rag.MetricWrapper)

Average Metric: 108.0 / 208  (51.9): 100%|██████████| 208/208 [1:02:13<00:00, 17.95s/it] 


,question,example_answer,context,pred_answer,MetricWrapper,answer
0,What is Student Ambassador Program at TBC College?,"The Student Ambassador Program at TBC College provides friendly and respectful peer support for students, utilizing the lived experience of international students to help build...","[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","The Student Ambassador Program at the referenced college is a peer support system for international students, providing assistance in various areas such as homesickness, academic...",✔️ [True],nan
1,How can I access more information about TBC College's Student Ambassador Program?,You can find more information by clicking on this link: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg,"[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","To access more information about TBC College's Student Ambassador Program, click on this link: [Student Ambassador Program](https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg). This link will provide you with detailed information...",✔️ [True],nan
2,What is TBC College's approach towards student support?,"TBC College provides peer support, which recognizes that students naturally turn to each other for support and connection. Student Ambassadors utilize their lived experience as...","[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","TBC College's approach towards student support is centered around peer-to-peer interaction, where Student Ambassadors, who are international students themselves, provide friendly and respectful support to...",✔️ [True],nan
3,How can I access more information about TBC College's student support program?,You can click here: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg for more details.,"[' Campus Facilities and Security Campus Security: As part of TBC commitment to safety, Security Guards are present on campus from 7.30 am to 9...","To access more information about TBC College's student support programs, you can visit their official website at [http://www.services.tbcollege.com](http://www.services.tbcollege.com). From there, navigate to the ""Services"" section...",False,nan
4,What tutoring services does TBC College offer for enrolled students?,TBC College offers free tutoring services to all enrolled students. These services could be peer-led or faculty-led.,"[' Tutoring Services Students who need help with their course material, LCIT offers free tutoring services to all enrolled students. Tutoring services could be peer-led...","Based on the provided information, it appears that TBC College does not explicitly mention any tutoring services for enrolled students. However, considering the context of...",False,nan
5,How can I request tutoring services at TBC College?,"To request tutoring services, please fill out the Tutor Request Form available here: https://forms.office.com/pages/responsepage.aspx?id=PlKz1i9IoEKTulrUSLtK5CmSQlqeXtROpTBCYMDn29lUMDBRQjYxMkhWSlkzOFBRS1NGR1RaNVQ1RSQlQCN0PWcu","[' Tutoring Services Students who need help with their course material, LCIT offers free tutoring services to all enrolled students. Tutoring services could be peer-led...","To request tutoring services at TBC College, follow these steps: 1. Visit the college's official website (http://www.tbcollege.com). 2. Navigate to the ""Services"" section in the...",✔️ [True],nan
6,How can I register with Accessibility Services at TBC College?,"To register with Accessibility Services, please fill out the Accessibility Service Application available at this link: https://forms.office.c

51.92

In [22]:
# Save model parameters

compiled_rag.save(params["rag_config"])

[('generate_answer', Predict(StringSignature(context, question -> rationale, answer
    instructions='Answer the questions directly using clear and concise language and do not provide extra details. If the answer is not found in the context, state: "For the above information, kindly reach out to academics or student support.\n    '
    context = Field(annotation=str required=True json_schema_extra={'desc': 'Context that contains all the relevant facts needed to answer the question.', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'desc': 'The question asked to the system.', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'Answer in 